In [181]:
import pandas as pd
import numpy.polynomial
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm

P21_2014 = pd.read_csv(r'D:\rprocious\PSM Lighthouse\CBI\Gap Filling\P21_2016.csv')

P21_2014['date'] = pd.to_datetime(P21_2014['#date+time'])
P21_2014['022-pwl'].replace([-999, -99, 99, 'NA', 'RM'], np.nan, inplace=True)
P21_2014['022-bwl'].replace([-999, -99, 99, 'NA', 'RM'], np.nan, inplace=True)
P21_2014['022-bwl'] = pd.to_numeric(P21_2014['022-bwl'],errors= 'coerce')
P21_2014['022-pwl'] = pd.to_numeric(P21_2014['022-pwl'],errors= 'coerce')


C:\Users\rprocious\AppData\Local\Temp\ipykernel_23196\1696970078.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  P21_2014['022-pwl'].replace([-999, -99, 99, 'NA', 'RM'], np.nan, inplace=True)
C:\Users\rprocious\AppData\Local\Temp\ipykernel_23196\1696970078.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

In [ ]:
def locate_gaps(WL_data):
    lengthMissVal = []
    dates = []
    count = 0
    for i in range(len(WL_data)):
        if pd.isna(WL_data['022-pwl'][i]):
            if count == 0:  # Start of a new NaN gap
                dates.append(WL_data['#date+time'][i])  # Record the start date of the gap
            count += 1  # Increment the gap length

        else:
            if count > 0:  # End of a NaN gap
                lengthMissVal.append(count)
                count = 0  # Reset count after recording the gap length

    # Finalize the DataFrame
    WL_data_gaps = pd.DataFrame()
    WL_data_gaps['date'] = dates
    WL_data_gaps['gapLength'] = lengthMissVal
    WL_data_gaps['gapTime(min)'] = WL_data_gaps['gapLength'] * 6
    return WL_data_gaps


def eligible_gap_length(WL_gaps): #Function to sort the lengh of the gaps into three categories
    WL_gaps_filter_6min = WL_gaps['gapLength'] == 1
    WL_gaps_filter_3hr = WL_gaps['gapLength'] <= 24
    WL_gaps_filter_3days = WL_gaps['gapLength'] <= 432

    #filters the data into individual dataframes
    linear_gaps = WL_gaps[WL_gaps_filter_6min]
    three_hr_gaps = WL_gaps[WL_gaps_filter_3hr]
    three_day_gaps = WL_gaps[WL_gaps_filter_3days]

    return linear_gaps,three_hr_gaps,three_day_gaps




def linear_fill(Wl_data,linear_gaps): #function to fill in gaps with length of 1 using linear approach

    matching_dates = Wl_data[Wl_data['date'].isin(linear_gaps['date'])]

    index_locations = matching_dates.index.tolist()

    for i in range(len(index_locations)):
        new_value = (Wl_data.loc[(index_locations[i])-1,'022-pwl']+ Wl_data.loc[index_locations[i]+1,'022-pwl']) / 2
        Wl_data.loc[index_locations[i],'022-pwl'] = new_value
    
    return Wl_data


def poly_gap_fill(Wl_data, three_hr_gaps):
    
    matching_dates = Wl_data[Wl_data['date'].isin(three_hr_gaps['date'])]

    index_locations = matching_dates.index.tolist()

    gap_length = three_hr_gaps['gapLength'].tolist()

    for i in range(len(index_locations)):

        if index_locations[i]- 2161  > 0 and index_locations[i]+2161+gap_length[i]:

            pwl_30_days = Wl_data['022-pwl'][(index_locations[i]- 2160):index_locations[i]+2160+gap_length[i]].tolist()

            bwl_30_days = Wl_data['022-bwl'][(index_locations[i]- 2160):index_locations[i]+2160+gap_length[i]].tolist()

            dates = Wl_data['date'][(index_locations[i]- 2160):index_locations[i]+2160+gap_length[i]].tolist()

            bwl_30_days_with_constants = sm.add_constant(bwl_30_days)

            bwl_30_days

            model = sm.OLS(pwl_30_days,bwl_30_days_with_constants,missing='drop')

            results = model.fit()

            slope = results.params[1]

            intercept = results.params[0]

            
            poly_df = pd.DataFrame({'bwl': bwl_30_days, 'pwl': pwl_30_days,'date' : pd.to_datetime(dates)})

            poly_df['mwl'] = intercept + slope*poly_df['bwl']

            poly_df.loc[abs(poly_df['mwl'] - poly_df['pwl']) > 0.1, ['mwl', 'pwl']] = np.nan

            


            
            if poly_df['bwl'].isna().sum() + poly_df['pwl'].isna().sum() < len(poly_df)*0.1:

                poly_df.dropna(inplace=True)

                poly =np.polynomial.polynomial.Polynomial.fit(poly_df['pwl'],poly_df['bwl'],4)

                pred_values = poly(poly_df['bwl'].values)

                poly_df['mwl'] = pred_values

            
        



Wl_gaps = locate_gaps(P21_2014)

linear_gaps,three_hr_gaps,three_day_gaps = eligible_gap_length(Wl_gaps)

linear_fill(P21_2014,linear_gaps)

poly_gap_fill(P21_2014,three_hr_gaps)



C:\Users\rprocious\AppData\Local\Temp\ipykernel_23196\2081290316.py:41: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  matching_dates = Wl_data[Wl_data['date'].isin(linear_gaps['date'])]
C:\Users\rprocious\AppData\Local\Temp\ipykernel_23196\2081290316.py:54: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  matching_dates = Wl_data[Wl_data['date'].isin(three_hr_gaps['date'])]


2.36680699 + 0.59142543 x - 0.00010757 x**2 - 0.00290029 x**3 -
0.00828555 x**4
        bwl    pwl                date       mwl
0     2.834  2.104 2016-01-08 22:30:00  3.289114
1     2.843  2.120 2016-01-08 22:36:00  3.293562
2     2.833  2.101 2016-01-08 22:42:00  3.288618
3     2.836  2.103 2016-01-08 22:48:00  3.290106
4     2.839  2.110 2016-01-08 22:54:00  3.291590
...     ...    ...                 ...       ...
4316  2.507  1.642 2016-01-26 22:06:00  3.104634
4317  2.493  1.624 2016-01-26 22:12:00  3.095907
4318  2.498  1.631 2016-01-26 22:18:00  3.099030
4319  2.508  1.644 2016-01-26 22:24:00  3.105256
4320  2.522  1.665 2016-01-26 22:30:00  3.113922

[4186 rows x 4 columns]
2.2876039 + 0.51233251 x + 0.0027458 x**2 + 0.00075535 x**3 -
0.00814625 x**4
        bwl    pwl                date       mwl
0     2.712  1.932 2016-01-09 22:12:00  3.183510
1     2.716  1.939 2016-01-09 22:18:00  3.185165
2     2.722  1.946 2016-01-09 22:24:00  3.187627
3     2.721  1.945 2016-01-09 22: